In [1]:
using JuMP, Cbc, Clp

# Newsvendor Problem

In [2]:
N = 1
d = collect(60:N:150)
p = (1/size(d,1))*ones(size(d,1));

In [3]:
model_Newsvendor = Model(with_optimizer(Cbc.Optimizer,logLevel=0))

@variables(model_Newsvendor,
begin
    x ≥ 0
    y[1:size(d,1)] ≥ 0
    w[1:size(d,1)] ≥ 0
end)

@constraints(model_Newsvendor,
begin
    ct1[s=1:size(d,1)], y[s] ≤ d[s]
    ct2[s=1:size(d,1)], y[s] + w[s] ≤ x
end)

@objective(model_Newsvendor, Min, 10 * x + sum(p[s]*(-25*y[s] -5*w[s]) for s = 1:size(d,1)))
optimize!(model_Newsvendor)
objective_value(model_Newsvendor)

Presolve 91 (-91) rows, 183 (0) columns and 273 (-91) elements
Perturbing problem by 0.001% of 10 - largest nonzero change 0.00017802805 ( 0.0020028054%) - largest zero change 0
0  Obj 0 Dual inf 29.999662 (182)
58  Obj -2.0329357e+10 Primal inf 3.7e+11 (37)
92  Obj -5.4944273e+09 Primal inf 9.9999999e+10 (10)
105  Obj -1404.3627
Optimal - objective value -1404.3956
After Postsolve, objective -1404.3956, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -1404.395604 - 105 iterations time 0.002, Presolve 0.00


-1404.3956043956057

In [4]:
value(x)

128.0

### Wait and See + EVPI

In [5]:
WS_Newsvendor_list = []
for dem in d
    mn = Model(with_optimizer(Cbc.Optimizer,logLevel=0))

    @variables(mn,
    begin
        x ≥ 0
        y ≥ 0
        w ≥ 0
    end)

    @constraints(mn,
    begin
        ct1, y ≤ dem
        ct2, y + w ≤ x
    end)

    @objective(mn, Min, 10 * x + (-25*y -5*w))

    optimize!(mn)
    append!(WS_Newsvendor_list, objective_value(mn))
end;

Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Optimal - objective value -900
After Postsolve, objective -900, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -900 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Optimal - objective value -915
After Postsolve, objective -915, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -915 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Optimal - objective value -930
After Postsolve, objective -930, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -930 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Optimal - objective value -945
After Postsolve, objective -945, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -945 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Optimal - objective value -960


In [27]:
WS_Newsvendor = sum(WS_Newsvendor_list)/size(d,1)

-1575.0

In [23]:
EVPI_Newsvendor = -WS_Newsvendor+objective_value(model_Newsvendor)

170.6043956043943

### VSS

In [8]:
x_ = 105
EEV_Newsvendor_list = []
for dem in d
    mn2 = Model(with_optimizer(Cbc.Optimizer,logLevel=0))

    @variables(mn2,
    begin
        y ≥ 0
        w ≥ 0
    end)

    @constraints(mn2,
    begin
        ct1, y ≤ dem
        ct2, y + w ≤ x_
    end)

    @objective(mn2, Min, 10 * x_ + (-25*y -5*w))

    optimize!(mn2)
    append!(EEV_Newsvendor_list, objective_value(mn2))
end;

Presolve 0 (-2) rows, 0 (-2) columns and 0 (-3) elements
Optimal - objective value -1725
After Postsolve, objective -1725, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -1725 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-2) columns and 0 (-3) elements
Optimal - objective value -1745
After Postsolve, objective -1745, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -1745 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-2) columns and 0 (-3) elements
Optimal - objective value -1765
After Postsolve, objective -1765, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -1765 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-2) columns and 0 (-3) elements
Optimal - objective value -1785
After Postsolve, objective -1785, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -1785 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-2) columns and 0 (-3) elements
Optimal - objective

In [9]:
EEV_Newsvendor = sum(EEV_Newsvendor_list)/size(d,1)

-1347.5274725274726

In [10]:
VSS_Newsvendor = -objective_value(model_Newsvendor)+EEV_Newsvendor

56.868131868133105

# Farmer Problem

In [11]:
plantacao = ["Wheat", "Corn", "Sugar Cane"]
produtos = ["Wheat", "Corn", "Sugar Cane High", "Sugar Cane Low"]
cenarios = [1, 2, 3]
preco_venda = [170, 150, 36, 10]
preco_producao = [150, 230, 260]
producao = [ 3    2.5  2  ;
             3.6  3    2.4;
            24   20   16]
preco_compra = [238, 210]
qtd_exigida = [200, 240]
terra_disponivel = 500
max_sugar_cane_high = 6000;

In [12]:
model_Farmer = Model(Clp.Optimizer)

@variables(model_Farmer,
begin
    X[plantacao] >= 0
    W[produtos, cenarios] >= 0
    Y[produtos[1:2], cenarios] >= 0
end)


@constraints(model_Farmer,
begin
    LAND_AVAILABLE, sum(X[pl] for pl in plantacao) <= terra_disponivel
    HERD_REQUIREMENTS[(ipr, pr) in enumerate(produtos[1:2]), k in cenarios], producao[ipr, k] * X[pr] + Y[pr, k] - W[pr, k] >= qtd_exigida[ipr]
    SUGAR_CANE_IMPOSITIONS_1[k in cenarios], sum(W[pr,k] for pr in produtos[3:4]) <= producao[3,k] * X["Sugar Cane"]
    SUGAR_CANE_IMPOSITIONS_2[k in cenarios], sum(W[produtos[3],k]) <= max_sugar_cane_high
end)

@objective(model_Farmer, Min, sum(preco_producao[ipl] * X[pl] for (ipl, pl) in enumerate(plantacao))
                 + sum((1/3)*(
                 + sum(preco_compra[ipr] * Y[pr, k] for (ipr, pr) in enumerate(produtos[1:2]))
                 - sum(preco_venda[ipr] * W[pr, k] for (ipr, pr) in enumerate(produtos))
                  ) for k in cenarios)
            )

optimize!(model_Farmer)
objective_value(model_Farmer)

Coin0506I Presolve 9 (-4) rows, 20 (-1) columns and 30 (-3) elements
Clp0006I 0  Obj 0 Primal inf 3643.2432 (7) Dual inf 452.72063 (11)
Clp0006I 13  Obj -108390
Clp0000I Optimal - objective value -108390
Coin0511I After Postsolve, objective -108390, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -108390 - 13 iterations time 0.002, Presolve 0.00


-108390.0

In [13]:
value.(X)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["Wheat", "Corn", "Sugar Cane"]
And data, a 3-element Vector{Float64}:
 170.0
  80.00000000000001
 250.0

In [14]:
value.(Y)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["Wheat", "Corn"]
    Dimension 2, [1, 2, 3]
And data, a 2×3 Matrix{Float64}:
 0.0  0.0   0.0
 0.0  0.0  48.0

In [15]:
value.(W)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["Wheat", "Corn", "Sugar Cane High", "Sugar Cane Low"]
    Dimension 2, [1, 2, 3]
And data, a 4×3 Matrix{Float64}:
  310.0   225.0   140.0
   48.0     0.0     0.0
 6000.0  5000.0  4000.0
    0.0     0.0     0.0

### Wait and See + EVPI

In [16]:
WS_Farmer_list = []
for c in cenarios
    
    mf = Model(Clp.Optimizer)
    @variables(mf,
    begin
        X[plantacao] >= 0
        W[produtos] >= 0
        Y[produtos[1:2]] >= 0
    end)


    @constraints(mf,
    begin
        LAND_AVAILABLE, sum(X[pl] for pl in plantacao) <= terra_disponivel
        HERD_REQUIREMENTS[(ipr, pr) in enumerate(produtos[1:2])], producao[ipr,c] * X[pr] + Y[pr] - W[pr] >= qtd_exigida[ipr]
        SUGAR_CANE_IMPOSITIONS_1, sum(W[pr] for pr in produtos[3:4]) <= producao[3,c] * X["Sugar Cane"]
        SUGAR_CANE_IMPOSITIONS_2, sum(W[produtos[3]]) <= max_sugar_cane_high
    end)

    @objective(mf, Min, sum(preco_producao[ipl] * X[pl] for (ipl, pl) in enumerate(plantacao))
                     + sum(preco_compra[ipr] * Y[pr] for (ipr, pr) in enumerate(produtos[1:2]))
                     - sum(preco_venda[ipr] * W[pr] for (ipr, pr) in enumerate(produtos))
                      )

    optimize!(mf)
    append!(WS_Farmer_list, objective_value(mf))
end;

Coin0506I Presolve 3 (-2) rows, 7 (-2) columns and 9 (-4) elements
Clp0006I 0  Obj 0 Primal inf 633.33333 (3) Dual inf 1654 (3)
Clp0006I 4  Obj -167666.67
Clp0000I Optimal - objective value -167666.67
Coin0511I After Postsolve, objective -167666.67, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -167666.6667 - 4 iterations time 0.002, Presolve 0.00
Coin0506I Presolve 3 (-2) rows, 7 (-2) columns and 9 (-4) elements
Clp0006I 0  Obj 0 Primal inf 660 (3) Dual inf 1335 (3)
Clp0006I 4  Obj -118600
Clp0000I Optimal - objective value -118600
Coin0511I After Postsolve, objective -118600, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -118600 - 4 iterations time 0.002, Presolve 0.00
Coin0506I Presolve 3 (-2) rows, 7 (-2) columns and 9 (-4) elements
Clp0006I 0  Obj 0 Primal inf 700 (3) Dual inf 1016 (3)
Clp0006I 5  Obj -59950
Clp0000I Optimal - objective value -59950
Coin0511I After Postsolve, objective -59950, infeasibilities - dual 0 (0), primal 0 (

In [17]:
WS_Farmer = sum(WS_Farmer_list)/size(cenarios,1)

-115405.55555555556

In [18]:
EVPI_Farmer = -WS_Farmer + objective_value(model_Farmer)

7015.555555555562

### VSS

In [19]:
EEV_Farmer_list = []
X_1 = Dict("Wheat" => 120, "Corn" => 80, "Sugar Cane" => 300)

for c in cenarios
    
    mf2 = Model(Clp.Optimizer)
    @variables(mf2,
    begin
        W[produtos] >= 0
        Y[produtos[1:2]] >= 0
    end)


    @constraints(mf2,
    begin
        LAND_AVAILABLE, sum(X_1[pl] for pl in plantacao) <= terra_disponivel
        HERD_REQUIREMENTS[(ipr, pr) in enumerate(produtos[1:2])], producao[ipr,c] * X_1[pr] + Y[pr] - W[pr] >= qtd_exigida[ipr]
        SUGAR_CANE_IMPOSITIONS_1, sum(W[pr] for pr in produtos[3:4]) <= producao[3,c] * X_1["Sugar Cane"]
        SUGAR_CANE_IMPOSITIONS_2, sum(W[produtos[3]]) <= max_sugar_cane_high
    end)

    @objective(mf2, Min, sum(preco_producao[ipl] * X_1[pl] for (ipl, pl) in enumerate(plantacao))
                     + sum(preco_compra[ipr] * Y[pr] for (ipr, pr) in enumerate(produtos[1:2]))
                     - sum(preco_venda[ipr] * W[pr] for (ipr, pr) in enumerate(produtos))
                      )

    optimize!(mf2)
    append!(EEV_Farmer_list, objective_value(mf2))
end;

Coin0506I Presolve 2 (-3) rows, 4 (-2) columns and 4 (-3) elements
Clp0006I 0  Obj -113600 Dual inf 320 (2)
Clp0006I 2  Obj -148000
Clp0000I Optimal - objective value -148000
Coin0511I After Postsolve, objective -148000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -148000 - 2 iterations time 0.002, Presolve 0.00
Coin0506I Presolve 2 (-3) rows, 4 (-2) columns and 4 (-3) elements
Clp0006I 0  Obj -101600 Dual inf 320 (2)
Clp0006I 2  Obj -118600
Clp0000I Optimal - objective value -118600
Coin0511I After Postsolve, objective -118600, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -118600 - 2 iterations time 0.002, Presolve 0.00
Coin0506I Presolve 1 (-4) rows, 2 (-4) columns and 2 (-5) elements
Clp0006I 0  Obj -48320 Dual inf 170 (1)
Clp0006I 1  Obj -55120
Clp0000I Optimal - objective value -55120
Coin0511I After Postsolve, objective -55120, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -55120 - 1 iterations time 0.002,

In [20]:
EEV_Farmer = sum(EEV_Farmer_list)/size(cenarios,1)

-107240.0

In [21]:
VSS_Farmer = -objective_value(model_Farmer)+EEV_Farmer

1150.0